## TETRAHEDRAL CONFIGURATION
### The following configuration allows one to experiment with different mic radii. The distance between the mic array sphere and it's closest 6 sources will be preserved, as will the distance between sources. 

In [ ]:
### VISUALIZE
# The top of the box is currently removed for the sake of visualization.

import trimesh
import numpy as np
from rlr_audio_propagation import Config, Context, ChannelLayout, ChannelLayoutType
import matplotlib.pyplot as plt
import soundfile as sf

width, depth, height = 7.0, 7.0, 3.0 
vertices = np.array([
    [0, 0, 0], [width, 0, 0], [width, depth, 0], [0, depth, 0],
    [0, 0, height], [width, 0, height], [width, depth, height], [0, depth, height]
])
faces = np.array([
    [0, 1, 2], [0, 2, 3], 
    [0, 4, 7], [0, 7, 3], 
    [1, 5, 6], [1, 6, 2], 
    [3, 2, 6], [3, 6, 7],  
    [0, 1, 5], [0, 5, 4]  
])

def spherical_to_cartesian(r, theta, phi):
    theta_rad = np.radians(theta)
    phi_rad = np.radians(phi)
    x = r * np.sin(theta_rad) * np.cos(phi_rad)
    y = r * np.sin(theta_rad) * np.sin(phi_rad)
    z = r * np.cos(theta_rad)
    return x, y, z

mic_radius = 0.17
mic_positions = [
    (55, 45),
    (125, 315),
    (125, 135),
    (55, 225)
]
mic_cartesian = [spherical_to_cartesian(mic_radius, theta, phi) for theta, phi in mic_positions]
room_center = [width/2, depth/2, height/2]

mic_meshes = []
mic_absolute_positions = []
for i, (x, y, z) in enumerate(mic_cartesian):
    mic_pos = [room_center[0] + x, room_center[1] + y, room_center[2] + z]
    mic_absolute_positions.append(mic_pos)
    mic_mesh = trimesh.creation.icosphere(radius=0.01, subdivisions=2)
    mic_mesh.apply_translation(mic_pos)
    mic_mesh.visual.face_colors = [255, 0, 0, 255] 
    mic_meshes.append(mic_mesh)

cfg = Config()
cfg.indirect_ray_count = 50000 
cfg.indirect_ray_depth = 25  
cfg.source_ray_count = 50000 
cfg.source_ray_depth = 25 
cfg.max_diffraction_order = 1
cfg.direct_ray_count = 8000 
cfg.max_ir_length = 1.5 
cfg.mesh_simplification = False

ctx = Context(cfg)
ctx.add_object()
ctx.set_object_position(0, [0, 0, 0])
ctx.add_mesh_vertices(vertices.flatten().tolist())
ctx.add_mesh_indices(faces.flatten().tolist(), 3, "default")
ctx.finalize_object_mesh(0)

for i, mic_pos in enumerate(mic_absolute_positions):
    ctx.add_listener(ChannelLayout(ChannelLayoutType.Mono, 1))
    ctx.set_listener_position(i, mic_pos)

closest_distances = {
    'X': 0.4031,
    'Y': 0.4031,
    'Z': 0.3500
}

distances_between_sources = {
    'X': 1.0,
    'Y': 1.0,
    'Z': 0.4000
}


closest_sources = [
    (room_center[0] + mic_radius + closest_distances['X'], room_center[1], room_center[2]),
    (room_center[0] - mic_radius - closest_distances['X'], room_center[1], room_center[2]),
    (room_center[0], room_center[1] + mic_radius + closest_distances['Y'], room_center[2]),
    (room_center[0], room_center[1] - mic_radius - closest_distances['Y'], room_center[2]),
    (room_center[0], room_center[1], room_center[2] + mic_radius + closest_distances['Z']),
    (room_center[0], room_center[1], room_center[2] - mic_radius - closest_distances['Z'])
]

source_positions = closest_sources.copy()

source_x1 = (source_positions[0][0] + distances_between_sources['X'], source_positions[0][1], source_positions[0][2])
source_x2 = (source_x1[0] + distances_between_sources['X'], source_x1[1], source_x1[2])
source_negx1 = (source_positions[1][0] - distances_between_sources['X'], source_positions[1][1], source_positions[1][2])
source_negx2 = (source_negx1[0] - distances_between_sources['X'], source_negx1[1], source_negx1[2])

source_y1 = (source_positions[2][0], source_positions[2][1] + distances_between_sources['Y'], source_positions[2][2])
source_y2 = (source_y1[0], source_y1[1] + distances_between_sources['Y'], source_y1[2])
source_negy1 = (source_positions[3][0], source_positions[3][1] - distances_between_sources['Y'], source_positions[3][2])
source_negy2 = (source_negy1[0], source_negy1[1] - distances_between_sources['Y'], source_negy1[2])

source_x11 = (source_x1[0], source_y1[1], source_x1[2])
source_x12 = (source_x1[0], source_y2[1], source_x1[2])
source_negx11 = (source_x1[0], source_negy1[1], source_x1[2])
source_negx12 = (source_x1[0], source_negy2[1], source_x1[2])

source_x21 = (source_x2[0], source_y1[1], source_x2[2])
source_x22 = (source_x2[0], source_y2[1], source_x2[2])
source_negx21 = (source_x2[0], source_negy1[1], source_x2[2])
source_negx22 = (source_x2[0], source_negy2[1], source_x2[2])

source_nx11 = (source_negx1[0], source_y1[1], source_negx1[2])
source_nx12 = (source_negx1[0], source_y2[1], source_negx1[2])
source_nnx11 = (source_negx1[0], source_negy1[1], source_negx1[2])
source_nnx12 = (source_negx1[0], source_negy2[1], source_negx1[2])

source_nx21 = (source_negx2[0], source_y1[1], source_negx2[2]) 
source_nx22 = (source_negx2[0], source_y2[1], source_negx2[2]) 
source_nnx21 = (source_negx2[0], source_negy1[1], source_negx2[2])
source_nnx22 = (source_negx2[0], source_negy2[1], source_negx2[2]) 


source_positions.extend([
    source_x1, source_x2, source_negx1, source_negx2,
    source_y1, source_y2, source_negy1, source_negy2,
    source_x11, source_x12, source_negx11, source_negx12, 
    source_x21, source_x22, source_negx21, source_negx22, 
    source_nx11, source_nx12, source_nnx11, source_nnx12, 
    source_nx21, source_nx22, source_nnx21, source_nnx22
])

# Build out z dimension
for i in range(6, 30):
    sourcez_1pos = (source_positions[i][0], source_positions[i][1], source_positions[i][2] + distances_between_sources['Z'])
    sourcez_2pos = (source_positions[i][0], source_positions[i][1], source_positions[i][2] + (2 * distances_between_sources['Z']))
    sourcez_1neg = (source_positions[i][0], source_positions[i][1], source_positions[i][2] - distances_between_sources['Z'])
    sourcez_2neg = (source_positions[i][0], source_positions[i][1], source_positions[i][2] - (2 * distances_between_sources['Z']))
    source_positions.extend([sourcez_1pos, sourcez_2pos, sourcez_1neg, sourcez_2neg])

source_centerzup = (source_positions[4][0], source_positions[4][1], source_positions[4][2] + distances_between_sources['Z']) 
source_centerzdown = (source_positions[5][0], source_positions[5][1], source_positions[5][2] - distances_between_sources['Z'])
source_positions.extend([source_centerzup, source_centerzdown])

# Create room mesh and add sources
box_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
for i, position in enumerate(source_positions):
    ctx.add_source()
    ctx.set_source_position(i, position)
    
    source_sphere = trimesh.creation.icosphere(radius=0.02, subdivisions=2)
    source_sphere.apply_translation(position)
    source_sphere.visual.face_colors = [0, 255, 0, 255]
    box_mesh = trimesh.util.concatenate([box_mesh, source_sphere])

# Combine all meshes
combined_mesh = trimesh.util.concatenate([box_mesh] + mic_meshes)
print(f"\nIs the mesh watertight? {combined_mesh.is_watertight}")
print(f"Number of sources: {len(source_positions)}")

print("\nSource Positions:")
for i, position in enumerate(source_positions):
    print(f"Source {i}: ({position[0]:.4f}, {position[1]:.4f}, {position[2]:.4f})")

combined_mesh.show()

In [37]:
### DATA GENERATION 

import os
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

wav_dir = f"Wavs_{mic_radius}"
plot_dir = f"Plots_{mic_radius}"
os.makedirs(wav_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

ctx = Context(cfg)
ctx.add_object()
ctx.set_object_position(0, [0, 0, 0])
ctx.add_mesh_vertices(vertices.flatten().tolist())
ctx.add_mesh_indices(faces.flatten().tolist(), 3, "default")
ctx.finalize_object_mesh(0)
ctx.add_listener(ChannelLayout(ChannelLayoutType.Mono, 1))
ctx.add_source()

for source_index, source_position in enumerate(source_positions):
    print(f"\nProcessing source {source_index + 1} at position {source_position}")
    
    # Calculate source's position relative to room center for filename
    relative_position = np.array(source_position) - np.array(room_center)
    x, y, z = relative_position
    coord_filename = f"{source_index:03d}_{x:.2f}_{y:.2f}_{z:.2f}"
    output_filename = os.path.join(wav_dir, f"{coord_filename}.wav")
    plot_filename = os.path.join(plot_dir, f"{coord_filename}.png")
    
    print(f"Output WAV file: {output_filename}")
    print(f"Output plot file: {plot_filename}")
    
    ir_all_mics = []
    ir_lengths = []
    
    # Process one microphone at a time
    for mic_index in range(4):
        print(f"  Processing microphone {mic_index + 1}")
        
        mic_position = np.array(mic_absolute_positions[mic_index])
        ctx.set_listener_position(0, mic_position.tolist())
        ctx.set_source_position(0, source_position)
        ctx.simulate()
        
        # write each mic to 1 channel of IR 
        channel = np.array(ctx.get_ir_channel(0, 0, 0))
        ir_all_mics.append(channel)
        ir_lengths.append(len(channel))
        print(f"    IR length: {len(channel)} samples")
    
    max_length = max(ir_lengths)
    print(f"  Max IR length: {max_length} samples")
    
    # Pad shorter IRs with zeros to match longest IR
    ir_all_mics_padded = [np.pad(ir, (0, max_length - len(ir)), 'constant') for ir in ir_all_mics]
    ir_all_mics = np.array(ir_all_mics_padded)
    
    sample_rate = int(cfg.sample_rate)
    print(f"  Writing WAV file with sample rate: {sample_rate} Hz")
    sf.write(output_filename, ir_all_mics.T, sample_rate)
    
    print("  Generating plot...")
    plt.figure(figsize=(15, 10))
    for i in range(4):
        plt.subplot(4, 1, i+1)
        plt.plot(ir_all_mics[i])
        plt.title(f'Microphone {i+1}')
        plt.ylim([-1, 1])
        plt.ylabel('Amplitude')
        plt.grid(True)
    plt.xlabel('Sample')
    plt.suptitle(f'Room Impulse Response - Tetrahedral Microphone Channels\nSource {source_index:03d}: {x:.2f}, {y:.2f}, {z:.2f}', fontsize=16)
    plt.tight_layout()
    plt.savefig(plot_filename)
    plt.close()
    print("  Plot saved.")

print(f"\nAll Room Impulse Responses saved in {wav_dir}")
print(f"All plots saved in {plot_dir}")

Created directories: Tetrahedral_wavs_0.17 and Tetrahedral_plots_0.17

Processing source 1 at position (4.0731, 3.5, 1.5)
Output WAV file: Tetrahedral_wavs_0.17/000_0.57_0.00_0.00.wav
Output plot file: Tetrahedral_plots_0.17/000_0.57_0.00_0.00.png
  Processing microphone 1
    Simulating...
    IR length: 12314 samples
  Processing microphone 2
    Simulating...
    IR length: 12368 samples
  Processing microphone 3
    Simulating...
    IR length: 13396 samples
  Processing microphone 4
    Simulating...
    IR length: 14579 samples
  Max IR length: 14579 samples


Material for category 'default' was not found. Using default material instead.


  Writing WAV file with sample rate: 44100 Hz
  Generating plot...
  Plot saved.

Processing source 2 at position (2.9269, 3.5, 1.5)
Output WAV file: Tetrahedral_wavs_0.17/001_-0.57_0.00_0.00.wav
Output plot file: Tetrahedral_plots_0.17/001_-0.57_0.00_0.00.png
  Processing microphone 1
    Simulating...
    IR length: 14670 samples
  Processing microphone 2
    Simulating...
    IR length: 14765 samples
  Processing microphone 3
    Simulating...
    IR length: 14741 samples
  Processing microphone 4
    Simulating...
    IR length: 14741 samples
  Max IR length: 14765 samples
  Writing WAV file with sample rate: 44100 Hz
  Generating plot...
  Plot saved.

Processing source 3 at position (3.5, 4.0731, 1.5)
Output WAV file: Tetrahedral_wavs_0.17/002_0.00_0.57_0.00.wav
Output plot file: Tetrahedral_plots_0.17/002_0.00_0.57_0.00.png
  Processing microphone 1
    Simulating...
    IR length: 14750 samples
  Processing microphone 2
    Simulating...
    IR length: 14774 samples
  Processin